In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from datetime import datetime

In [3]:
# Load the data
with open('./data/technical_data.pkl', 'rb') as f:
    data = pd.read_pickle(f)

# Add 'Make' column for each brand
for make, records in data.items():
    for record in records:
        record['Make'] = make

# Combine all records into a single DataFrame
data_df = []
for make, records in data.items():
    data_df.extend(records)
data_df = pd.DataFrame(data_df)

In [4]:
data_df.to_csv('./data/raw_data.csv', index=False)

In [ ]:
data_df.head()

In [ ]:
data_df.info()

In [ ]:
data_df.describe()

In [5]:
# Clean Numeric Columns
def clean_numeric_column(column, remove_text=True):
    if remove_text:
        return column.str.replace(r'[^\d.]', '', regex=True).astype(float)
    return column

In [6]:
# Convert  columns to numeric
data_df["price"] = clean_numeric_column(data_df["price"])
data_df["Mileage"] = clean_numeric_column(data_df["Mileage"])
data_df["Cubic Capacity"] = clean_numeric_column(data_df["Cubic Capacity"])
data_df["Power"] = data_df["Power"].str.extract(r"(\d+)", expand=False).astype(float)

In [7]:
data_df.to_csv('./data/raw_data_cast.csv', index=False)

In [ ]:
# Calculate the proportion of missing values
missing_values = (data_df.isnull().sum() / len(data_df)) * 100
missing_values = missing_values[missing_values > 0]  # get columns with missing values

# Plot the missing values
plt.figure(figsize=(12, 8))
sns.barplot(
    x=missing_values.values,
    y=missing_values.index,
    palette="viridis",
    hue=missing_values.values,
)
plt.title("Proportion of Missing Values by Column")
plt.xlabel("Percentage of Missing Values")
plt.ylabel("Columns")
plt.show()

In [ ]:
# Drop columns with more than 50% missing values
threshold = 50
columns_to_drop = missing_values[missing_values > threshold].index.tolist()
print(f"Columns with more than {threshold}% missing values: {columns_to_drop}")

data_df_cleaned = data_df.drop(columns=columns_to_drop, axis=1)

print("\nCleaned Data Information:")
data_df_cleaned.info()

In [ ]:
# Get unique values for categorical columns
categorical_columns = data_df_cleaned.select_dtypes(include="object").columns
for col in categorical_columns:
    unique_values = data_df_cleaned[col].nunique()
    print(f"{col}: {unique_values} unique values")

# PLot
plt.figure(figsize=(16, 8))
for i, col in enumerate(["Make", "Fuel", "Transmission", "Drive type"], start=1):
    plt.subplot(2, 2, i)
    sns.countplot(
        y=data_df_cleaned[col],
        order=data_df_cleaned[col].value_counts().index,
        palette="viridis",
        hue=data_df_cleaned[col],
    )
    plt.title(f"Distribution of {col}")
    plt.xlabel("Count")
    plt.ylabel(col)

plt.tight_layout()
plt.show()

In [ ]:
# Missing value imputation for important columns
data_df_cleaned["Cubic Capacity"].fillna(
    data_df_cleaned["Cubic Capacity"].median(), inplace=True
)
data_df_cleaned["Power"].fillna(data_df_cleaned["Power"].median(), inplace=True)
data_df_cleaned["Number of Seats"].fillna(
    data_df_cleaned["Number of Seats"].mode()[0], inplace=True
)

# Drop rows with missing values in critical columns (e.g., 'price', 'Mileage')
data_df_cleaned = data_df_cleaned.dropna(subset=["price", "Mileage"])

# Confirm that missing values are handled
print("\nRemaining Missing Values:")
print(data_df_cleaned.isnull().sum())

In [ ]:
# Boxplots to identify outliers
numeric_columns = ["price", "Mileage", "Cubic Capacity", "Power"]
plt.figure(figsize=(16, 8))
for i, col in enumerate(numeric_columns, start=1):
    plt.subplot(2, 2, i)
    sns.boxplot(data=data_df_cleaned, x=col)
    plt.title(f"Boxplot of {col}")
    plt.xlabel(col)

plt.tight_layout()
plt.show()

# Handle outliers using the IQR method
for col in ["price", "Mileage"]:
    Q1 = data_df_cleaned[col].quantile(0.25)
    Q3 = data_df_cleaned[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    data_df_cleaned = data_df_cleaned[
        (data_df_cleaned[col] >= lower_bound) & (data_df_cleaned[col] <= upper_bound)
        ]

print("\nDataset after cleaning:")
data_df_cleaned.info()

In [13]:
# Save the cleaned dataset to a CSV file
data_df_cleaned.to_csv('./data/cleaned_data.csv', index=False)

## Univariate Analysis

In [ ]:
# Univariate Analysis for Key Numeric Features
plt.figure(figsize=(16, 8))

# Price Distribution
plt.subplot(2, 2, 1)
sns.histplot(data_df_cleaned['price'], bins=50, kde=True, color='blue')
plt.title("Distribution of Price")
plt.xlabel("Price (€)")
plt.ylabel("Frequency")

# Mileage Distribution
plt.subplot(2, 2, 2)
sns.histplot(data_df_cleaned['Mileage'], bins=50, kde=True, color='orange')
plt.title("Distribution of Mileage")
plt.xlabel("Mileage (km)")
plt.ylabel("Frequency")

# Power Distribution
plt.subplot(2, 2, 3)
sns.histplot(data_df_cleaned['Power'], bins=50, kde=True, color='green')
plt.title("Distribution of Power")
plt.xlabel("Power (kW)")
plt.ylabel("Frequency")

# Cubic Capacity Distribution
plt.subplot(2, 2, 4)
sns.histplot(data_df_cleaned['Cubic Capacity'], bins=50, kde=True, color='purple')
plt.title("Distribution of Cubic Capacity")
plt.xlabel("Cubic Capacity (cc)")
plt.ylabel("Frequency")

plt.tight_layout()
plt.show()


In [ ]:
# Univariate Analysis for Key Categorical Features
plt.figure(figsize=(16, 8))

# Make Distribution
plt.subplot(2, 2, 1)
sns.countplot(data_df_cleaned['Make'], order=data_df_cleaned['Make'].value_counts().index)
plt.title("Distribution of Car Makes")
plt.xlabel("Make")
plt.ylabel("Count")
plt.xticks(rotation=45)

# Fuel Distribution
plt.subplot(2, 2, 2)
sns.countplot(data_df_cleaned['Fuel'], order=data_df_cleaned['Fuel'].value_counts().index)
plt.title("Distribution of Fuel Types")
plt.xlabel("Fuel Type")
plt.ylabel("Count")
plt.xticks(rotation=45)

# Transmission Distribution
plt.subplot(2, 2, 3)
sns.countplot(data_df_cleaned['Transmission'], order=data_df_cleaned['Transmission'].value_counts().index)
plt.title("Distribution of Transmission Types")
plt.xlabel("Transmission")
plt.ylabel("Count")
plt.xticks(rotation=45)

# Drive Type Distribution
plt.subplot(2, 2, 4)
sns.countplot(data_df_cleaned['Drive type'], order=data_df_cleaned['Drive type'].value_counts().index)
plt.title("Distribution of Drive Types")
plt.xlabel("Drive Type")
plt.ylabel("Count")
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

## Conclusions from Univariate Analysis
1. Price
    * The price distribution is normal, with most vehicles priced around €20,000.
    * A small number of luxury or high-end vehicles create outliers at the higher end.

2. Mileage
    * Mileage follows a somewhat bell-shaped curve, with most vehicles between 50,000 km and 140,000 km.
    * Few vehicles have extremely low or high mileage, but they may represent unique cases (e.g., new or heavily used vehicles).

3. Power
    * The power distribution shows a clustering around 100–200 kW, which is common for German vehicles.
    * Vehicles with very high power are rare, likely reflecting high-performance or specialty cars.

4. Cubic Capacity
    * The cubic capacity distribution peaks around 1,500–2,000 cc, which aligns with typical engine sizes for German cars.
    * Larger engines (e.g., above 3,000 cc) are less frequent and likely associated with premium or performance cars.

## Bivariate Analysis
### Purpose:
   To explore relationships between pairs of variables, such as:

* price vs. Mileage
* price vs. Power
* price vs. categorical variables like Make or Fuel

In [ ]:
# Bivariate Analysis: Numeric vs. Numeric
plt.figure(figsize=(16, 8))

# Price vs. Mileage
plt.subplot(2, 2, 1)
sns.scatterplot(data=data_df_cleaned, x='Mileage', y='price', alpha=0.6, color='blue')
plt.title("Price vs. Mileage")
plt.xlabel("Mileage (km)")
plt.ylabel("Price (€)")

# Price vs. Power
plt.subplot(2, 2, 2)
sns.scatterplot(data=data_df_cleaned, x='Power', y='price', alpha=0.6, color='orange')
plt.title("Price vs. Power")
plt.xlabel("Power (kW)")
plt.ylabel("Price (€)")

# Price vs. Cubic Capacity
plt.subplot(2, 2, 3)
sns.scatterplot(data=data_df_cleaned, x='Cubic Capacity', y='price', alpha=0.6, color='green')
plt.title("Price vs. Cubic Capacity")
plt.xlabel("Cubic Capacity (cc)")
plt.ylabel("Price (€)")

plt.tight_layout()
plt.show()

# Price vs. Categorical Variables
plt.figure(figsize=(16, 8))

# Price by Make
plt.subplot(2, 2, 1)
sns.boxplot(data=data_df_cleaned, x='price', y='Make', palette="viridis", showfliers=False, hue='Make')
plt.title("Price by Make")
plt.xlabel("Price (€)")
plt.ylabel("Make")

# Price by Fuel
plt.subplot(2, 2, 2)
sns.boxplot(data=data_df_cleaned, x='price', y='Fuel', palette="magma", showfliers=False, hue='Fuel')
plt.title("Price by Fuel Type")
plt.xlabel("Price (€)")
plt.ylabel("Fuel Type")

# Price by Transmission
plt.subplot(2, 2, 3)
sns.boxplot(data=data_df_cleaned, x='price', y='Transmission', palette="plasma", showfliers=False, hue='Transmission')
plt.title("Price by Transmission Type")
plt.xlabel("Price (€)")
plt.ylabel("Transmission")

plt.tight_layout()
plt.show()


## Correlation Analysis
### Purpose:
To identify linear relationships between numeric variables and the target variable (price).

In [ ]:
# Correlation Matrix
numeric_cols = ['price', 'Mileage', 'Cubic Capacity', 'Power']
correlation_matrix = data_df_cleaned[numeric_cols].corr()

# Heatmap of Correlations
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Correlation Matrix")
plt.show()


## Multivariate Analysis
### Purpose:
Explore how multiple variables interact, e.g., the effect of Make and Fuel type on price.

In [ ]:
# Pairplot for selected features
selected_features = ['price', 'Mileage', 'Power', 'Cubic Capacity']
sns.pairplot(data_df_cleaned[selected_features], diag_kind='kde', corner=True)
plt.show()

In [ ]:
# Boxplot with two categorical variables
plt.figure(figsize=(12, 8))
sns.boxplot(data=data_df_cleaned, x='Fuel', y='price', hue='Transmission', palette='viridis', showfliers=False)
plt.title("Price by Fuel Type and Transmission")
plt.xlabel("Fuel Type")
plt.ylabel("Price (€)")
plt.legend(title="Transmission")
plt.show()

## Conclusions
1. Bivariate Analysis

* Price vs. Mileage:
    * Clear negative relationship: cars with higher mileage generally have lower prices.
    * A few outliers exist (e.g., low-price cars with very low mileage).
* Price vs. Power:
    * Positive relationship: cars with higher power (kW) tend to have higher prices.
    * The trend weakens for vehicles with extremely high power.
* Price vs. Cubic Capacity:
    * Moderate positive relationship: larger engines generally correlate with higher prices.
    * A notable clustering around common cubic capacities (e.g., ~2,000 cc).

2. Price by Categorical Features

* Price by Make:
    * Luxury brands like Audi and BMW have higher median prices compared to Skoda and Volkswagen.
    * Price variance is highest for Audi, reflecting its diverse product range.
* Price by Fuel Type:
    * Diesel cars tend to have slightly higher prices compared to petrol cars.
    * Alternative fuels (e.g., hybrid) appear less frequently but with higher prices.
* Price by Transmission:
    * Automatic vehicles have higher median prices compared to manual ones.
    * Likely reflects demand and technology differences.

3. Correlation Analysis

* Strongest Correlations:
    * Power has a moderately strong positive correlation with price (~0.6).
    * Mileage has a moderate negative correlation with price (~-0.5).
    * Cubic Capacity shows a weaker positive correlation with price (~0.5).

* Key Insights:
    * Engine-related metrics (Power, Cubic Capacity) and Mileage are important predictors of price.
    * Multicollinearity among numeric variables appears limited, which is ideal for modeling.

## Feature Engineering

* Derived Features:
    * Vehicle Age: Extracted from the First Registration column.
    * Mileage per Year: Normalized mileage based on vehicle age.

* Encoded Features:
    * Categorical columns (Make, Fuel, Transmission, Drive type) were one-hot encoded.
    * Other categorical columns containing a high percentage of distinct values which are not relevant for the analysis (Interior Color, for example, we dropped)

* Transformed Features:
    * Log transformations applied to price (Log Price) and Mileage (Log Mileage) to handle skewness.

* Standardized Features:
    * Key numeric features (Mileage, Power, Cubic Capacity, Vehicle Age, Mileage per Year) were standardized to ensure consistent scaling.

In [ ]:
# Derive Features
current_year = datetime.now().year
data_df_cleaned['First Registration Year'] = data_df_cleaned['First Registration'].str.extract(r'(\d{4})').astype(float)
data_df_cleaned['Vehicle Age'] = current_year - data_df_cleaned['First Registration Year']
data_df_cleaned['Mileage per Year'] = data_df_cleaned['Mileage'] / data_df_cleaned['Vehicle Age']

# Encoding Categorical Variables
categorical_columns_to_encode = ['Vehicle condition', 'Make', 'Category', 'Door Count', 'Emission Class',
                                 'Climatisation', 'Parking sensors', 'Airbags', 'Colour', 'Fuel', 'Transmission',
                                 'Drive type']

# Perform one-hot encoding
encoder = OneHotEncoder(sparse_output=False)
encoded_columns = pd.DataFrame(
    encoder.fit_transform(data_df_cleaned[categorical_columns_to_encode]),
    columns=encoder.get_feature_names_out(categorical_columns_to_encode),
    index=data_df_cleaned.index
)

# Drop all remaining categorical columns
categorical_columns_to_drop = data_df_cleaned.select_dtypes(include='object').columns
data_df_encoded = pd.concat([data_df_cleaned.drop(columns=categorical_columns_to_drop), encoded_columns], axis=1)

# Step 3: Handle Skewness
data_df_encoded['Log Price'] = np.log1p(data_df_encoded['price'])
data_df_encoded['Log Mileage'] = np.log1p(data_df_encoded['Mileage'])

# Step 4: Standardization
numeric_features = ['Mileage', 'Power', 'Cubic Capacity', 'Vehicle Age', 'Mileage per Year']
scaler = StandardScaler()
data_df_encoded[numeric_features] = scaler.fit_transform(data_df_encoded[numeric_features])

print("Feature Engineering Summary:")
print(f"- Encoded Features: {encoder.get_feature_names_out(categorical_columns_to_encode).tolist()}")
print(f"- Dropped Columns: {categorical_columns_to_drop.tolist()}")
print("Transformed Dataset:")
print(data_df_encoded.info())


In [21]:
# Save the final dataset to a CSV file
data_df_encoded.to_csv('./data/processed_data.csv', index=False)